In [5]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
import ast
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from collections import Counter
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 20})



from sklearn.preprocessing import StandardScaler


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_plate_infos.pick"
global_plate_info = pd.read_pickle(path_save)


In [7]:
global_hypha_info['hypha_unique'] = global_hypha_info['hypha'].astype(str)+'_'+global_hypha_info['Plate'].astype(str)
time_hypha_info['hypha_unique'] = time_hypha_info['end'].astype(str)+'_'+time_hypha_info['Plate'].astype(str)

In [8]:
def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)

In [9]:
global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [59]:
plates = [94]

select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]

# select_global = select_global.loc[select_global['lapse']>2]
hyphae_interest= select_global['hypha_unique'].unique()
select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]
speed_at_init = select_time.loc[select_time['time_since_emergence']==1]

select_time = select_time.loc[select_time['hypha_unique'].isin(hyphae_interest)]
table = select_global.loc[select_global['hypha_unique'].isin(hyphae_interest)]
table = table.set_index('hypha_unique')
select_time = select_time.loc[select_time['speed'].between(50,400)]
select_time['speed'] = select_time['speed'].astype(float)
select_time = select_time.loc[select_time['distance_final_pos']>400]
select_time = select_time.loc[select_time['in_ROI']=='True']
hyphae = select_time.groupby("hypha_unique")["timestep"].count()>=2
table = table.loc[table.index.isin(hyphae.index)]
select_time = select_time.set_index("hypha_unique")
select_time = select_time.loc[hyphae[select_time.index]]
table['max_speed'] = select_time.groupby('hypha_unique')['speed'].max()
table['mean_speed2'] = select_time.groupby('hypha_unique')['speed'].mean()
table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
table['log_growth'] = np.log10((table['tot_growth_C']+1).astype(float))

table['growth_lapse'] = select_time.groupby('hypha_unique')['speed'].count()
table['speed_at_init'] = speed_at_init.groupby('hypha_unique')['speed'].max()

In [24]:
table['max_speed']

hypha_unique
0_94         350.241680
1_94         347.011704
2_94         209.114917
14_94        267.450869
19_94        371.905472
                ...    
113610_94    235.839422
114195_94    311.418135
115344_94           NaN
115356_94           NaN
118923_94           NaN
Name: max_speed, Length: 329, dtype: float64

In [53]:
from pandas.plotting import scatter_matrix
scatter_matrix(table[['av_width_final','log_length','mean_speed2','max_speed','log_growth']].fillna(0), alpha=0.15, figsize=(6, 6), diagonal="kde",s=70);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
table['is_rh'] = (table['log_length']>=3.36).astype(int)
table['is_small'] = (table['log_length']<=2.3).astype(int)
table['is_bas'] = (table['log_length'].between(2.3,3.36)).astype(int)


In [62]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
# ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('log_length','av_width_final',c='is_rh',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
# ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'94'}, xlabel='log_length', ylabel='av_width_final'>

Text(0, 0.5, 'average width $(\\mu m)$')

Text(0.5, 0, 'log length $(log(\\mu m))$')

In [63]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
# ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('log_length','max_speed',c='is_rh',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'94'}, xlabel='log_length', ylabel='max_speed'>

Text(0, 0.5, 'average width $(\\mu m)$')

Text(0.5, 0, 'log length $(log(\\mu m))$')

(50.0, 400.0)

In [64]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ylabel = 'average width $(\mu m)$'
# ylabel = 'maximum speed $(\mu m.s^{-1})$'
xlabel = 'log length $(log(\mu m))$'

table.plot('log_length','speed_at_init',c='is_rh',kind='scatter',colormap='Dark2',alpha = 0.2,s=100,ax=ax,title = plate,colorbar = False)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
# ax.set_xlim(2.5,4.5)
ax.set_ylim(50,400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'94'}, xlabel='log_length', ylabel='speed_at_init'>

Text(0, 0.5, 'average width $(\\mu m)$')

Text(0.5, 0, 'log length $(log(\\mu m))$')

(50.0, 400.0)